# 3webアプリ作成

目次：https://docs.google.com/document/d/1eDw9PK5Ft0vImu81e7oERB_q-dZvxbFe8bGetZBgEEg/edit

## 3.3 応用実装

### 3.3.1 応用アプリ実装の概要：５つのアプリを作る

### 3.3.2 応用アプリ実装

#### ②株価可視化アプリ

>アプリの概要

<img src="pic/img040.png">  

デプロイ先：https://kabu-step2-ver6-6ijpoom5ze3.streamlit.app/

>動作を構成してる部品の紹介（APIやライブラリ）
API
- yahoo finance
    - 目的：株価情報をyf.Ticker("銘柄番号")の１行で取得できる
    - 参考：https://pypi.org/project/yfinance/

ライブラリ
- streamlit
    - 目的：株価情報を表示するためのフロントエンドを生成する
- pandas
    - 目的：株価APIで得られたデータを扱いやすくするために使う
- yfinance
    - 目的：yahoo financeの株価情報取得手段をライブラリとして提供してくれている。
- altair
    - 目的：pandsで構造化データになった株価情報のデータ可視化のために使う



>ライブラリのインストール

ライブラリ
- streamlit
    - pip install streamlit
- pandas
    - pip install pandas
- yfinance
    - pip install yfinance
- altair
    - pip install altair

>ライブラリの公式ドキュメントの見方や使い方を説明


ライブラリ

- streamlit
    - インストール方法はget startページで確認
    - 参考：https://docs.streamlit.io/library/get-started/installation
    - st.titleなどはコンポーネントページから確認出来る
    - 参考：https://streamlit.io/components
- pandas
    - pd.DataFrame()で使います。
    - 参考：https://pandas.pydata.org/docs/
- yfinance
    - yf.Ticker("銘柄番号")でその銘柄の株価を取得できます。
    - 東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
    - 参考：https://pypi.org/project/yfinance/
- altair
    - alt.Chartでチャート生成
    - オプションは.encode()の中に設定していきます。
    - 参考：https://pypi.org/project/altair/

>それぞれの機能をjupyterで動く最小限を作る

以下がjuptyterにて動く最低限のアプリの流れです。

1. 作るアプリの全体のコードを把握
1. それぞれ機能を解説
    1. 株価取得
    1. チャートによる可視化

1.全体のコード

使うアプリの全体像は下記のようになります。  
これはweb上では動きませんが、手元では動作するアプリです。

以下のコードの動作の流れ
1. 拡張機能であるライブラリをインポート
1. 株価データを要求するための準備
    1. 株価を取得する会社すべてをtickersに代入
    1. 要求する日付区間をdaysに代入
1. 株価データ取得
    1. tickersを元にyf.Ticker("ここ銘柄")で１つ１つデータ取得
    1. dfにデータを格納
    1. dfにtickersにあるすべての企業の株価が格納されている
1. 株価データ整理
    1. 欲しい銘柄だけをcompaniesに代入
    1. companiesを使って取得した情報の中から情報を抽出
1. チャートで可視化
    1. チャートの表示範囲の値をyminとymaxに代入
    1. alt.Chartを使って抽出した株価データを可視化

In [22]:
!pip install yfinance


In [23]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
    'AMADA': '6113.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示

days = 10 # 取得するための日数daysに代入

print(f"過去 {days}日間 の株価") # 取得する日数を表示


def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = ['google', 'apple','TOYOTA']

data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
print("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)


# チャートに表示する範囲、それぞれをymin, ymaxに代入
print("株価の範囲指定") # サイドバーに表示
ymin = 0
ymax = 3000

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)

AAPL: No price data found, symbol may be delisted (period=10d)


株価可視化アプリ
こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。
表示日数選択
過去 10日間 の株価


AttributeError: 'Index' object has no attribute 'strftime'

2.それぞれ機能を解説

2.1株価取得

具体例として、TOYOTAを参考に株価を取得します。  
必要な情報は銘柄：7203.T だけです。

yf.Ticker(ここ銘柄)で情報を取得できます。  
可視化はtkr.history(period='ここ欲しい区間') で構造化データを取得できます。

In [ ]:
thr = yf.Ticker("7203.T") # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
hist = thr.history(period= "30d")
display(hist)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-02-28 00:00:00+09:00,3588.0,3599.0,3541.0,3570.0,21824800,0.0,0.0
2024-02-29 00:00:00+09:00,3560.0,3629.0,3540.0,3621.0,42143900,0.0,0.0
2024-03-01 00:00:00+09:00,3595.0,3689.0,3586.0,3680.0,29522200,0.0,0.0
2024-03-04 00:00:00+09:00,3694.0,3704.0,3636.0,3662.0,33112100,0.0,0.0
2024-03-05 00:00:00+09:00,3665.0,3742.0,3641.0,3729.0,31082800,0.0,0.0
2024-03-06 00:00:00+09:00,3729.0,3772.0,3690.0,3769.0,31920900,0.0,0.0
2024-03-07 00:00:00+09:00,3799.0,3811.0,3640.0,3660.0,44389400,0.0,0.0
2024-03-08 00:00:00+09:00,3644.0,3665.0,3595.0,3610.0,39889700,0.0,0.0
2024-03-11 00:00:00+09:00,3500.0,3530.0,3443.0,3498.0,34641400,0.0,0.0


数字だけでは、どの企業のデータかわからないので、カラムにTOYOTAの企業名を入れ、終値だけ取得します。

In [ ]:
# indexを日付のフォーマットに変更
# データを終値だけ抽出
# データのカラムをyf.Tickerのリクエストした企業名に設定
hist.index = hist.index.strftime('%d %B %Y')
hist = hist[['Close']]
hist.columns = ["TOYOTA"]
display(hist)


,TOYOTA
Date,
28 March 2024,3806.0
29 March 2024,3799.0
01 April 2024,3639.0
02 April 2024,3633.0
03 April 2024,3620.0
04 April 2024,3667.0
05 April 2024,3619.0
08 April 2024,3698.0
09 April 2024,3776.0


注意点として、複数の株価データが日数が違うデータもindexであるDateに順番に格納されてしまうので、  
対策として日付をカラムにします。

.Tで転置ができます。

In [ ]:
# 欲しい情報が逆なので、転置する
 # indexの名前をNameにする

hist = hist.T
hist.index.name = 'Name'
display(hist)

Date,2024-02-28 00:00:00+09:00,2024-02-29 00:00:00+09:00,2024-03-01 00:00:00+09:00,2024-03-04 00:00:00+09:00,2024-03-05 00:00:00+09:00,2024-03-06 00:00:00+09:00,2024-03-07 00:00:00+09:00,2024-03-08 00:00:00+09:00,2024-03-11 00:00:00+09:00,2024-03-12 00:00:00+09:00,...,2024-03-28 00:00:00+09:00,2024-03-29 00:00:00+09:00,2024-04-01 00:00:00+09:00,2024-04-02 00:00:00+09:00,2024-04-03 00:00:00+09:00,2024-04-04 00:00:00+09:00,2024-04-05 00:00:00+09:00,2024-04-08 00:00:00+09:00,2024-04-09 00:00:00+09:00,2024-04-10 00:00:00+09:00
Name,,,,,,,,,,,,,,,,,,,,,
Open,3588.0,3560.0,3595.0,3694.0,3665.0,3729.0,3799.0,3644.0,3500.0,3441.0,...,3839.0,3804.0,3801.0,3685.0,3633.0,3670.0,3605.0,3665.0,3740.0,3750.0
High,3599.0,3629.0,3689.0,3704.0,3742.0,3772.0,3811.0,3665.0,3530.0,3475.0,...,3854.0,3833.0,3824.0,3705.0,3656.0,3739.0,3632.0,3700.0,3776.0,3760.0
Low,3541.0,3540.0,3586.0,3636.0,3641.0,3690.0,3640.0,3595.0,3443.0,3398.0,...,3782.0,3792.0,3616.0,3616.0,3597.0,3652.0,3568.0,3642.0,3716.0,3722.0
Close,3570.0,3621.0,3680.0,3662.0,3729.0,3769.0,3660.0,3610.0,3498.0,3475.0,...,3806.0,3799.0,3639.0,3633.0,3620.0,3667.0,3619.0,3698.0,3776.0,3740.0
Volume,21824800.0,42143900.0,29522200.0,33112100.0,31082800.0,31920900.0,44389400.0,39889700.0,34641400.0,40384800.0,...,34646300.0,12094300.0,38828100.0,31273800.0,30222400.0,33637800.0,26966800.0,23272600.0,22518400.0,17062700.0
Dividends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stock Splits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


これで株価データの取得が完了しました。

2.2チャートによる可視化

チャートにするために、構造化データを日付をx軸、株価をy軸に変更します。
複数のデータがあった方がわかりやすいので、チャートのための構造化データを以下のコードで作成します。

In [ ]:
# tickersに設定した銘柄一覧でリクエストする一覧を設定
 # 株価を代入するための空箱を用意

 # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
    # スライドバーで指定した日数で取得した情報を絞る
    # indexを日付のフォーマットに変更
    # データを終値だけ抽出
    # データのカラムをyf.Tickerのリクエストした企業名に設定
    # 欲しい情報が逆なので、転置する
     # indexの名前をNameにする
    # 用意した空のデータフレームに設定したhistのデータを結合する

tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
    'AMADA': '6113.T',
}
days = 10

df = pd.DataFrame()

for company in tickers.keys():
    thr = yf.Ticker(tickers[company])
    hist = thr.history(period=f'{days}d')
    hist.index = hist.index.strftime('%d %B %Y')
    hist = hist[['Close']]
    hist.columns = [company]
    hist = hist.T
    hist.index.name ='Name'
    df = pd.concat([df, hist])

display(df)

Date,26 March 2024,27 March 2024,28 March 2024,01 April 2024,02 April 2024,03 April 2024,04 April 2024,05 April 2024,08 April 2024,09 April 2024,29 March 2024,10 April 2024
Name,,,,,,,,,,,,
apple,169.710007,173.309998,171.479996,170.029999,168.839996,169.649994,168.820007,169.580002,168.449997,169.669998,NaN,NaN
facebook,495.890015,493.859985,485.579987,491.350006,497.369995,506.739990,510.920013,527.340027,519.250000,516.900024,NaN,NaN
google,150.669998,150.869995,150.929993,155.490005,154.559998,154.919998,150.529999,152.500000,154.850006,156.600006,NaN,NaN
microsoft,421.649994,421.429993,420.720001,424.570007,421.440002,420.450012,417.880005,425.519989,424.589996,426.279999,NaN,NaN
netflix,629.239990,613.530029,607.330017,614.309998,614.210022,630.080017,617.140015,636.179993,628.409973,618.200012,NaN,NaN
amazon,178.300003,179.830002,180.380005,180.970001,180.690002,182.410004,180.000000,185.070007,185.190002,185.669998,NaN,NaN
TOTO,NaN,NaN,4236.000000,4221.000000,4163.000000,4092.000000,4115.000000,4212.000000,4296.000000,4325.000000,4271.0,4335.0
TOYOTA,NaN,NaN,3806.000000,3639.000000,3633.000000,3620.000000,3667.000000,3619.000000,3698.000000,3776.000000,3799.0,3740.0
AMADA,NaN,NaN,1724.500000,1688.000000,1683.500000,1678.000000,1695.500000,1673.500000,1669.500000,1705.500000,1721.5,1708.5


構造化データを日付をx軸、株価をy軸するために転置します。

In [ ]:
# 全体像のコードと一致させるために変数名を変更します。
 # dataを抽出して転置

data = df
data = data.T.reset_index()
display(data)


Name,Date,apple,facebook,google,microsoft,netflix,amazon,TOTO,TOYOTA,AMADA
0,26 March 2024,169.710007,495.890015,150.669998,421.649994,629.239990,178.300003,NaN,NaN,NaN
1,27 March 2024,173.309998,493.859985,150.869995,421.429993,613.530029,179.830002,NaN,NaN,NaN
2,28 March 2024,171.479996,485.579987,150.929993,420.720001,607.330017,180.380005,4236.0,3806.0,1724.5
3,01 April 2024,170.029999,491.350006,155.490005,424.570007,614.309998,180.970001,4221.0,3639.0,1688.0
4,02 April 2024,168.839996,497.369995,154.559998,421.440002,614.210022,180.690002,4163.0,3633.0,1683.5
5,03 April 2024,169.649994,506.739990,154.919998,420.450012,630.080017,182.410004,4092.0,3620.0,1678.0
6,04 April 2024,168.820007,510.920013,150.529999,417.880005,617.140015,180.000000,4115.0,3667.0,1695.5
7,05 April 2024,169.580002,527.340027,152.500000,425.519989,636.179993,185.070007,4212.0,3619.0,1673.5
8,08 April 2024,168.449997,519.250000,154.850006,424.589996,628.409973,185.190002,4296.0,3698.0,1669.5
9,09 April 2024,169.669998,516.900024,156.600006,426.279999,618.200012,185.669998,4325.0,3776.0,1705.5


バラバラのデータである必要がないので、企業を１つにまとめます。  
使い方は以下のようになります。

pd.melt(ここにデータ, id_vars=[変更した良くない列]).rename(
    columns={'value': '新しいカラムの名前'}
)

In [ ]:
# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)
display(data)


,Date,Name,Stock Prices
0,26 March 2024,apple,169.710007
1,27 March 2024,apple,173.309998
2,28 March 2024,apple,171.479996
3,01 April 2024,apple,170.029999
4,02 April 2024,apple,168.839996
...,...,...,...
103,05 April 2024,AMADA,1673.500000
104,08 April 2024,AMADA,1669.500000
105,09 April 2024,AMADA,1705.500000
106,29 March 2024,AMADA,1721.500000


2.2チャートによる可視化

alt.Chartを使って、チャートデータを作成します。  
指定している設定は以下のようになります。

- チャートに使うデータをdata
- xのラベルにDate　色種類は:T
- yのラベルにStock Prices　色種類は:Q　範囲0, 3000
- 全体の色種類指定:N

In [ ]:
# chartで表示するデータを入れる

# 作成したチャートをstreamlitで表示

chart = (
    alt.Chart(data).mark_line(opacity=0.8, clip=True).encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale= alt.Scale(domain = [0, 5000])),
        color='Name:N'
    )
)

display(chart)

alt.Chart(...)

以上がアプリの動作部分です。

>streamlitに実装する場合の使う機能

今はユーザーがブラウザ上で選択したりなど、想定していませんが、  
選択肢の表示などのコンポーネントを紹介します。

※streamlitはjupyterでは動作せず、実行ファイルでのみ動作します

タイトルは  
st.title("ここにタイトル")

ただの表示は  
st.write("ここに表示したい文字")

ただし、再読み込みしない限り、この表示された文字は変更されません。

選択肢は  st.selectbox("ここに表示したい項目名",ここに配列のデータ)  
でできます。

サイドバーに配置したい場合は、   
st.slider.write("ここに表示したい文字")のように.sliderをstのうしろにつけます。

スライドバーの表示は  
st.sidebar.slider('ここに表示したい文字', 最小値の数字, 最大値の数字, 初期の位置の値)  
でできます。

チャートの表示は  
st.altair_chart(ここにちゃーとのデータ, use_container_width=True)でできます。

以上の機能を使ってwebアプリにしていきます。

>（演習）streamlitに実装しましょう！

手元で動いたアプリをstreamlitで動くようにします。  
この演習では必ず実行ファイルにコーディングしてください。

※streamlit run ファイル名で実行する時は必ず実行ファイルで実行してください。

実装の流れ
1. 取得した全株価から抽出する条件companiesの内容をセレクターで取得
    1. 構造化データから選択肢抽出
    1. companiesにセレクトボックスの値が入るようにする
1. 固定数字をスライドバーで変数へ
1. ターミナルに出力していた文字などをブラウザに出力

以下の手元で動いたコードをstreamlitで動くようにします。

In [ ]:
# フロントエンドを扱うstreamlitの機能をインポート
# データフレームを扱う機能をインポート
# yahoo financeから株価情報を取得するための機能をインポート
# チャート可視化機能をインポート

import streamlit as st
import pandas as pd
import yfinance as yf
import altair as alt

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる

tickers = {
      'apple': 'AAPL',
      'facebook': 'META',
      'google': 'GOOGL',
      'microsoft': 'MSFT',
      'netflix': 'NFLX',
      'amazon': 'AMZN',
      'TOTO': '5332.T',
      'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示


# 取得するための日数daysに代入

 # 取得する日数を表示

days = 10

print(f"過去 {days}日間 の株価") 

 # 株価を代入するための空箱を用意

 

def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = ['google', 'apple','TOYOTA']


data =df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
 # dataにあるindexを表示
 # dataを抽出して転置

print("株価" , data.sort_index()) # dataを抽出して転置
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# チャートに表示する範囲、それぞれをymin, ymaxに代入
# サイドバーに表示
ymin = 0
ymax = 3000

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示

display(chart)

株価可視化アプリ
こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。
表示日数選択
過去 10日間 の株価
株価 Date    26 March 2024  27 March 2024  28 March 2024  01 April 2024  \
Name                                                                 
TOYOTA            NaN            NaN    3806.000000    3639.000000   
apple      169.710007     173.309998     171.479996     170.029999   
google     150.669998     150.869995     150.929993     155.490005   

Date    02 April 2024  03 April 2024  04 April 2024  05 April 2024  \
Name                                                                 
TOYOTA    3633.000000    3620.000000    3667.000000    3619.000000   
apple      168.839996     169.649994     168.820007     169.580002   
google     154.559998     154.919998     150.529999     152.500000   

Date    08 April 2024  09 April 2024  29 March 2024  10 April 2024  
Name                                                                
TOYOTA    3698.000000    3776.000000         3799.0         3740.0  
apple      168.449997 

alt.Chart(...)

1.取得した全株価から抽出する条件companiesの内容をセレクターで取得

df.indexで構造化データから取得した企業のリストを取得できるので、これをlist関数を使って配列にする。

In [ ]:
list(df.index)

['apple',
 'facebook',
 'google',
 'microsoft',
 'netflix',
 'amazon',
 'TOTO',
 'TOYOTA']

セレクトボックスの値をcompaniesに代入する。

使い方は  
st.selectbox("ここに表示したい項目名",ここに配列のデータ,初期で選択されている配列)

初期で選択されている配列はどれでも構いませんが、ここでは['google', 'apple','TOYOTA']を指定しておきます。

In [24]:
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'],
)

2.固定数字をスライドバーで変数へ

10日で固定しているdaysとチャートの表示範囲ymin,ymaxをスライドバーから取得します。  
変更する固定数字は以下の３つです。

In [ ]:
days = 10

ymin = 0
ymax = 3000

スライドバーの使い方は以下のようになります。  
st.slider('ここに表示したい文字', 最小値の数字, 最大値の数字, 初期の位置の値) 

また１つのスライドバーから２つの値を取得する場合は、

小さい方の数字, 大きいの方の数字 = st.sidebar.slider(
    'ここに表示したい文字',
    最小値の数字, 最大値の数字, (最小値の数字, 最大値の数字)
)

でできます。

In [ ]:
days = st.sidebar.slider('日数', 1, 50, 20)

ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
)

streamlitに実装したアプリをまとめると、以下のようになります。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

print('株価可視化アプリ')
print("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。")# サイドバーに表示
print("表示日数選択") # サイドバーに表示

days = st.sidebar.slider('日数', 1, 50, 20) # 取得するための日数daysに代入

print(f"過去 {days}日間 の株価") # 取得する日数を表示


def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得





# 企業名の配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'],
)

data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
print("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)


# チャートに表示する範囲、それぞれをymin, ymaxに代入
print("株価の範囲指定") # サイドバーに表示


ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
display(chart)

株価可視化アプリ
こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。
表示日数選択
過去 20日間 の株価
株価  Date    11 March 2024  12 March 2024  13 March 2024  14 March 2024  \
Name                                                                 
TOYOTA            NaN    3475.000000    3444.000000    3445.000000   
apple      172.750000     173.229996     171.130005     173.000000   
google     137.669998     138.500000     139.789993     143.100006   

Date    15 March 2024  18 March 2024  19 March 2024  20 March 2024  \
Name                                                                 
TOYOTA    3488.000000    3567.000000    3675.000000            NaN   
apple      172.619995     173.720001     176.080002     178.669998   
google     141.179993     147.679993     147.029999     148.740005   

Date    21 March 2024  22 March 2024  ...  27 March 2024  28 March 2024  \
Name                                  ...                                 
TOYOTA    3799.000000    3872.000000  ...    3853.000000    3806.000000   
app

alt.Chart(...)

3.ターミナルに出力していた文字などをブラウザに出力

最後にprintでターミナルで出力していた  
タイトルや文字やチャートを以下の４つのコードを使ってブラウザに出力します。

- st.title
- st.write
- st.altair_chart

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

st.title('株価可視化アプリ') # タイトル

st.write("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。") # サイドバーに表示

st.write("表示日数選択") # サイドバーに表示

days = st.slider('日数', 1, 50, 20) # サイドバーに表示　取得するための日数をスライドバーで表示し、daysに代入

st.write(f"過去 {days}日間 の株価") # 取得する日数を表示

# @st.cache_dataで読み込みが早くなるように処理を保持しておける
@st.cache_data
def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

# チャートに表示する範囲をスライドで表示し、それぞれをymin, ymaxに代入
st.write("株価の範囲指定") # サイドバーに表示
ymin, ymax = st.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
) # サイドバーに表示

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得


# 取得したデータから抽出するための配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'], # 最初に表示する企業名を設定
)


data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
st.write("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
st.altair_chart(chart, use_container_width=True)

最後にサイドバーに表示する項目に.sidebarをつけます。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import pandas as pd # データフレームを扱う機能をインポート
import yfinance as yf # yahoo financeから株価情報を取得するための機能をインポート
import altair as alt # チャート可視化機能をインポート

#取得する銘柄の名前とキーを変換する一覧を設定
#東証などのシンボルはhttps://support.yahoo-net.jp/PccFinance/s/article/H000006603で検索できる
tickers = {
    'apple': 'AAPL',
    'facebook': 'META',
    'google': 'GOOGL',
    'microsoft': 'MSFT',
    'netflix': 'NFLX',
    'amazon': 'AMZN',
    'TOTO': '5332.T',
    'TOYOTA': '7203.T',
}

st.title('株価可視化アプリ') # タイトル

st.sidebar.write("こちらは株価可視化ツールです。以下のオプションから表示日数を指定できます。") # サイドバーに表示

st.sidebar.write("表示日数選択") # サイドバーに表示

days = st.sidebar.slider('日数', 1, 50, 20) # サイドバーに表示　取得するための日数をスライドバーで表示し、daysに代入

st.write(f"過去 {days}日間 の株価") # 取得する日数を表示

# @st.cache_dataで読み込みが早くなるように処理を保持しておける
@st.cache_data
def get_data(days, tickers):
    df = pd.DataFrame() # 株価を代入するための空箱を用意

    # 選択した株価の数だけ yf.Tickerでリクエストしてデータを取得する
    for company in tickers.keys():
        tkr = yf.Ticker(tickers[company]) # 設定した銘柄一覧でリクエストの為の7203.Tなどに変換をして、それをyf.Tickerで株価リクエスト
        hist = tkr.history(period=f'{days}d') # スライドバーで指定した日数で取得した情報を絞る
        hist.index = hist.index.strftime('%d %B %Y') # indexを日付のフォーマットに変更
        hist = hist[['Close']] # データを終値だけ抽出
        hist.columns = [company] # データのカラムをyf.Tickerのリクエストした企業名に設定
        hist = hist.T # 欲しい情報が逆なので、転置する
        hist.index.name = 'Name' # indexの名前をNameにする
        df = pd.concat([df, hist]) # 用意した空のデータフレームに設定したhistのデータを結合する
    return df # 返り値としてdfを返す

# チャートに表示する範囲をスライドで表示し、それぞれをymin, ymaxに代入
st.sidebar.write("株価の範囲指定") # サイドバーに表示
ymin, ymax = st.sidebar.slider(
    '範囲を指定してください。',
    0.0, 5000.0, (0.0, 5000.0)
) # サイドバーに表示

df = get_data(days, tickers) # リクエストする企業一覧すべてと変換するtickersを引数に株価取得


# 取得したデータから抽出するための配列を生成し、companiesに代入
companies = st.multiselect(
    '会社名を選択してください。',
    list(df.index),
    ['google', 'apple','TOYOTA'], # 最初に表示する企業名を設定
)


data = df.loc[companies] # 取得したデータから抽出するための配列で絞ってdataに代入
st.write("株価 ", data.sort_index()) # dataにあるindexを表示
data = data.T.reset_index() # dataを抽出して転置

# 企業ごとの別々のカラムにデータを表示する必要ないので企業を１つのカラムに統一
data = pd.melt(data, id_vars=['Date']).rename(
    columns={'value': 'Stock Prices'}
)

# dataとスライドバーで設定した最大最小値を元にalt.Chartを使って株価チャートを作成
chart = (
    alt.Chart(data)
    .mark_line(opacity=0.8, clip=True)
    .encode(
        x="Date:T",
        y=alt.Y("Stock Prices:Q", stack=None, scale=alt.Scale(domain=[ymin, ymax])),
        color='Name:N'
    )
)

# 作成したチャートをstreamlitで表示
st.altair_chart(chart, use_container_width=True)